In [5]:
import sys, os, decimal, json
sys.path.append(os.path.realpath('..'))
import json, time
import pandas as pd
import matplotlib.pyplot as plt

from TrainingPipelines.ImageClassifier.ModelBuilder import build_and_train

from google.cloud import storage
import os
from pathlib import Path
from dotenv import load_dotenv

env_path = Path(fr"{os.environ['USERPROFILE']}\.env")
load_dotenv(dotenv_path=env_path)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("Eve_Online_GCP_Creds_File")

import seaborn as sns
import mplcyberpunk

plt.style.use("cyberpunk")
plt.rcParams['figure.facecolor'] = '#0d1117'
plt.rcParams['axes.facecolor'] = '#0d1117'
plt.rcParams['savefig.facecolor'] = '#0d1117'

In [6]:
def render_cm(stats):
    df_cm = pd.DataFrame(stats['cm'])
    plt.figure(figsize = (10,10))
    sns.heatmap(df_cm, annot=True)

def write_rendering(file, rendering_dict):
    json_object = json.dumps(rendering_dict, indent=1)
    with open(file, "w") as outfile:
        outfile.write(json_object)    

def save_to_gcp(source_file, dest_file):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket("eve_online_models") # your bucket name
    blob = bucket.blob(dest_file) 
    blob.upload_from_filename(source_file)
    #blob = bucket.blob('eve_online_models/Eve_Online_Models/test.h5') 
    #blob.upload_from_filename(r"O:\eve_models\training_data\game_state\game_state_model.h5")

In [ ]:
model_name = 'route_y_large_vert_class_v2'
training_root = r'O:\eve_models\training_data\route_y_large_vert_class_v2'
model_file = f'{training_root}/{model_name}_model.h5'
meta_file = f'{training_root}/{model_name}_meta.json'

params = {
    'root_image_directory': training_root,
    'model_location': model_file,
    'model_name': model_name,
    'epochs':30, 
    'resize_ratio':1
}

stats, rendering = build_and_train(**params)
render_cm(stats)
write_rendering(meta_file, rendering)
print(rendering)

save_to_gcp(source_file=model_file, dest_file=f'{model_name}/{model_name}_model.h5')
save_to_gcp(source_file=meta_file, dest_file=f'{model_name}/{model_name}_meta.json')

1755
Found 1755 files belonging to 25 classes.
Using 1404 files for training.
Found 1755 files belonging to 25 classes.
Using 351 files for validation.
['200', '210', '220', '250', '260', '270', '290', '310', '320', '330', '340', '350', '360', '370', '380', '390', '400', '410', '420', '430', '440', '460', '470', '480', '490']
Epoch 1/30
1404/1404 [==============================] - 12s 7ms/step - loss: 1.0468 - accuracy: 0.8034 - val_loss: 0.3581 - val_accuracy: 0.9088
Epoch 2/30
1404/1404 [==============================] - 10s 7ms/step - loss: 0.4485 - accuracy: 0.8989 - val_loss: 0.2487 - val_accuracy: 0.9430
Epoch 3/30
1404/1404 [==============================] - 10s 7ms/step - loss: 0.3596 - accuracy: 0.9195 - val_loss: 0.2327 - val_accuracy: 0.9430
Epoch 4/30
1397/1404 [============================>.] - ETA: 0s - loss: 0.3304 - accuracy: 0.9105

In [ ]:
model_name = 'set_dest'
training_root = r'O:\eve_models\training_data\set_dest'
model_file = f'{training_root}/{model_name}_model.h5'
meta_file = f'{training_root}/{model_name}_meta.json'

params = {
    'root_image_directory': training_root,
    'model_location': model_file,
    'model_name': model_name
}

stats, rendering = build_and_train(**params)
render_cm(stats)
write_rendering(meta_file, rendering)
print(rendering)

save_to_gcp(source_file=model_file, dest_file=f'{model_name}/{model_name}_model.h5')
save_to_gcp(source_file=meta_file, dest_file=f'{model_name}/{model_name}_meta.json')

In [ ]:
model_name = 'hanger_menus'
training_root = r'O:\eve_models\training_data\hanger_menus'
model_file = f'{training_root}/{model_name}_model.h5'
meta_file = f'{training_root}/{model_name}_meta.json'

params = {
    'root_image_directory': training_root,
    'model_location': model_file,
    'model_name': model_name
}

stats, rendering = build_and_train(**params)
render_cm(stats)
write_rendering(meta_file, rendering)
print(rendering)

save_to_gcp(source_file=model_file, dest_file=f'{model_name}/{model_name}_model.h5')
save_to_gcp(source_file=meta_file, dest_file=f'{model_name}/{model_name}_meta.json')

In [ ]:
model_name = 'nav_options'
training_root = r'O:\eve_models\training_data\nav_options'
model_file = f'{training_root}/{model_name}_model.h5'
meta_file = f'{training_root}/{model_name}_meta.json'

params = {
    'root_image_directory': training_root,
    'model_location': model_file,
    'model_name': model_name
}

stats, rendering = build_and_train(**params)
render_cm(stats)
write_rendering(meta_file, rendering)
print(rendering)

save_to_gcp(source_file=model_file, dest_file=f'{model_name}/{model_name}_model.h5')
save_to_gcp(source_file=meta_file, dest_file=f'{model_name}/{model_name}_meta.json')

In [ ]:
model_name = 'mining_tool_state'
training_root = r'O:\eve_models\training_data\mining_tool_state'
model_file = f'{training_root}/{model_name}_model.h5'
meta_file = f'{training_root}/{model_name}_meta.json'

params = {
    'root_image_directory': training_root,
    'model_location': model_file,
    'model_name': model_name
}

stats, rendering = build_and_train(**params)
render_cm(stats)
write_rendering(meta_file, rendering)
print(rendering)

save_to_gcp(source_file=model_file, dest_file=f'{model_name}/{model_name}_model.h5')
save_to_gcp(source_file=meta_file, dest_file=f'{model_name}/{model_name}_meta.json')

In [ ]:
model_name = 'game_state'
training_root = r'O:\eve_models\training_data\game_state'
model_file = f'{training_root}/{model_name}_model.h5'
meta_file = f'{training_root}/{model_name}_meta.json'

params = {
    'root_image_directory': training_root,
    'model_location': model_file,
    'model_name': model_name
}

stats, rendering = build_and_train(**params)
render_cm(stats)
write_rendering(meta_file, rendering)
print(rendering)

save_to_gcp(source_file=model_file, dest_file=f'{model_name}/{model_name}_model.h5')
save_to_gcp(source_file=meta_file, dest_file=f'{model_name}/{model_name}_meta.json')